# Projet 5 : Data analysis by combining a batch and a speed layer 



In [1]:
from datetime import date, datetime, timedelta
import time
import pandas as pd
from bson.json_util import dumps
from collections import Counter
import pymongo
import analyse #analyse.py

Connexion a MongoDB

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient.tweets
mycolbatch = mydb.batch_test2
mycolstream = mydb.streaming_test2

Recuperer les parametres pour definir l'intervalle de temps sur laquelle l'utiliateur cherche a avoir des donnees

In [3]:
rp1 = None
while rp1 is None:
    input_value = input("Voulez-vous recuperer les donnees depuis quelques dernieres heures? reponse Oui = O; Non = N ")
    try:
        
        if str(input_value).lower() == 'o' or str(input_value).lower() == 'n':
            rp1 = str(input_value)
            print('ok')
            
        else:
            print("{input} Veuillez preciser si vous voulez recuperer les donnees depuis quelques dernieres heures? reponse Oui = O; Non = N".format(input=input_value))
    except ValueError:
        print("{input} Veuillez preciser si vous voulez recuperer les donnees depuis quelques dernieres heures? reponse Oui = O; Non = N".format(input=input_value))
    

Voulez-vous recuperer les donnees depuis quelques dernieres heures? reponse Oui = O; Non = N n
ok


In [4]:
if rp1 == 'o':
    print ("Depuis combien d'heures voulez-vous recuperer des donnees? example: 3")
    nbh = input()
    nbh = int(nbh)
    startdate = datetime.now() - timedelta(hours = nbh)
    enddate = datetime.now()
    print("date debut: %s" %startdate)
    print("date fin: %s" %enddate)
else :  
    startdate = input("Veuillez renseigner une date debut sous format YYYY-MM-DD-HH (example 2020-07-27-15): ")
    year, month, day,hour = map(int, startdate.split('-'))
    startdate = datetime(year, month, day,hour)
    enddate = input("Veuillez renseigner une date de fin sous format YYYY-MM-DD-HH: ")
    year1, month1, day1,hour1 = map(int, enddate.split('-'))
    enddate = datetime(year1, month1, day1,hour1)
    print(startdate,enddate)

# creer une liste contenant toutes les tranches d'horaire; 1 tranche = 1 heure
time_range = pd.date_range(startdate, enddate, freq='H')    
dlist = []
for tmp in time_range:
    if len(str(tmp.hour)) > 1:
        h = str(tmp.hour)
    else:
        h = str(0) + str(tmp.hour)

    if len(str(tmp.day)) > 1:
        d = str(tmp.day)
    else:
        d = str(0) + str(tmp.day)
    if len(str(tmp.month)) > 1:
        m = str(tmp.month)
    else:
        m = str(0) + str(tmp.month)
    y = str(tmp.year)
    dlist.append(d+m+y+h)
print(dlist)

Veuillez renseigner une date debut sous format YYYY-MM-DD-HH (example 2020-07-27-15): 2020-07-28-15
Veuillez renseigner une date de fin sous format YYYY-MM-DD-HH: 2020-07-28-18
2020-07-28 15:00:00 2020-07-28 18:00:00
['2807202015', '2807202016', '2807202017', '2807202018']


Creer des requetes pour interroger les bases de donnees sur MongoDB ( batch and speed layer).

Le code doit permmettre aux 2 "layers" de se relayer pour satisfaire l'objectif d'une requete.

In [5]:
dbatch = {}
dstream = {}
for it in dlist:
    mybatchquery = {"name": it}
    mybatchdocJson = mycolbatch.find_one(mybatchquery)
    try:
        # si le donnees existent sur le batch layer
        if mybatchdocJson is not None:
            print("Batch layer: ")
            d1 = dict(mybatchdocJson)['hashktags']
            dmybatchdoc = {}
            for i in range(len(d1)):
                k = d1[i]['hashtags']
                v = d1[i]['count']
                dmybatchdoc[k] = v
            dbatch.update(dmybatchdoc)
            print("*******")
            print("founded at %s" %it)
        # si non, c'est la speed layer qui prend le relais
        else:
            print("========================================")
            print("Streaming layer: ")
            mystreamingquery = { "name": it}
            mystreamingdocJson = mycolstream.find_one(mystreamingquery)
            d2 = dict(mystreamingdocJson)['hashktags']
            dmystreamingdoc = {}
            for i in range(len(d2)):
                k = d2[i]['hashtags']
                v = d2[i]['count']
                dmystreamingdoc[k] = v
            dstream.update(dmystreamingdoc)
            print("*******")
            print("founded at %s" %it)
            
    except:
        print("No data exist")
        print("at %s" %it)
        
    

Batch layer: 
*******
founded at 2807202015
Batch layer: 
*******
founded at 2807202016
Streaming layer: 
*******
founded at 2807202017
Streaming layer: 
No data exist
at 2807202018


In [6]:
D = dict(Counter(dbatch) + Counter(dstream))
Dsorted = sorted(D.items() , reverse=True, key=lambda x: x[1])

In [7]:
# Iterate over the sorted sequence, keep the Top ten only
print("the 10 most tweeted hashtags:\n")
for elem in Dsorted[:20] :
    print(elem[0] , ":" , elem[1] )  

the 10 most tweeted hashtags:

ExaBFF : 412
ExaGOT7 : 238
ExaARMY : 214
GOT7 : 199
ImranKhanKhulwaoPUBG : 144
MTVHottest : 122
10000HoursbyJK : 100
1DWWATOUR : 90
BTS : 87
10000HoursWithJK : 84
JusticeForGloriaBambo : 83
RheaChakraborty : 77
MOTSPARTY : 74
WhyFearCBIForSSR : 69
JUNGKOOK : 62
नियोजन_नहीं_तो_वोट_नहीं : 60
TAEMIN : 59
COVID19 : 55
TrumpKillsTexas : 55
BarrHearing : 51


Using the module "analyse"

In [8]:
inp = analyse.TweetAnalyse.get_input()

Voulez-vous recuperer les donnees depuis quelques dernieres heures? reponse Oui = O; Non = N n
ok
Veuillez renseigner une date debut sous format YYYY-MM-DD-HH (example 2020-07-27-15): 2020-07-28-15
Veuillez renseigner une date de fin sous format YYYY-MM-DD-HH: 2020-07-28-18
2020-07-28 15:00:00 2020-07-28 18:00:00
['2807202015', '2807202016', '2807202017', '2807202018']
Combien de sujets les plus tweetes que vous voulez trouver? 10


In [9]:
inp.get_result()

Combining the results of the batch and the speed layer
Batch layer: 
*******
founded at 2807202015
Batch layer: 
*******
founded at 2807202016
Streaming layer: 
*******
founded at 2807202017
Streaming layer: 
No data exist
at 2807202018
Batch layer: 
*******
founded at 2807202015
Batch layer: 
*******
founded at 2807202016
Streaming layer: 
*******
founded at 2807202017
Streaming layer: 
No data exist
at 2807202018

The 10 most tweeted hashtags:

ExaBFF : 412
ExaGOT7 : 238
ExaARMY : 214
GOT7 : 199
ImranKhanKhulwaoPUBG : 144
MTVHottest : 122
10000HoursbyJK : 100
1DWWATOUR : 90
BTS : 87
10000HoursWithJK : 84
